In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 100)


In [ ]:
# notebook goal: Setup a basic machine learning framework that cleans data, standardizes features,
#  evaluates feature impt, shap values, and a myriad of ML algorithms
# TODO: add the day-of-week as a feature
# TODO: Add in target date versus historic reference dates
# TODO: Add in volume-based feature functionality
# TODO: Evaluate standardizing features per stock or one model per stock - may not be enough data realistically
# TODO: Check bol-range-pct calculation - only giving zero value

In [233]:
# functions:

def clean_stock_data(dataframe: pd.DataFrame) -> pd.DataFrame :

    '''removes nulls and in the future will be built out to do any additonal cleaning on the dataframe that is necessary
    Args:
        dataframe: pandas dataframe containing all of the potential features
        parameters: 
            calculation_field: field on which all of the features are built

    Returns:
        dataframe: dataset that is ready to load into a machine learning framework
    '''

    # remove records the preceed the target period to have complete information:
    dataframe.dropna(inplace = True)
    dataframe = dataframe.reset_index(drop = True)
    
    # remove fields that will not be used as predictive features (can be hardcoded since dataframe structure will be the same):
    dataframe = dataframe.drop(columns = [ 'date', 'ticker', 'high', 'low', 'open', 'volume', 'adj_close'])
    

    return dataframe


def identify_fields_to_standardize(dataframe: pd.DataFrame, parameters: dict) -> np.array :

    '''creates a list of the continuous fields to standardize by dimension within the predictive model; NOTE: this is used within the standardizer
    
    Args:
        dataframe: dataframe that contains all of the fields of interest to be used in the calculations
        parameters:
            continuous_feature_cutoff: ratio of unique values to record count to be used to codify continuous features

    Returns: list of continuous fields to use in the standardization process based on user's specifications of "uniqueness" threshold    

    '''

    numeric_fields = dataframe.select_dtypes(include = 'number').columns
    records = len(dataframe)

    record_summary = pd.DataFrame(dataframe[numeric_fields].nunique(), columns = ['unique_values'])
    record_summary['rows_in_df'] = records
    record_summary['value_to_record_ratio'] = record_summary['unique_values']/ record_summary['rows_in_df']

    # filter for a threshold specified by the user:
    record_summary = record_summary[record_summary['value_to_record_ratio'] > parameters['continuous_feature_cutoff']]

    # remove percentage features # TODO: later add in functionality to remove percentage based features

    return record_summary.index


# Justification for approach on scaling - the argument can be made that since our approach will generalize movemements across multiple securities that we need to standardize each security to its own price range.  Therefore, any features with price-relative values will be scaled per the security's price values to avoid odd splits in tree-based algos
# the concern with standardization is generally focused on not letting any one feature have considerably more weight in a model than another; however in this case, 


def standardize_continuous_features(dataframe: pd.DataFrame, parameters: dict) -> pd.DataFrame:

    '''function that identifies the continuious features in the dataframe and standardizes each feature by equity to enable scaling relative to each equity
    
    Args:
        Dataframe: Pandas dataframe to be used in machine learning
        Parameters:
            stock_field: field indicating the stock for the window function to scan
            calculation_field: field for which the target is being calculated (used for drop in main row merge)
    
    Returns:
        Dataframe: containing the standardized data fields
    
    '''

    continuous_fields = list(identify_fields_to_standardize(dataframe = dataframe, parameters = parameters))

    # add in the ticker for grouping next:
    continuous_fields.append(parameters['stock_field'])

    # downselect to the fields that will be used to standardize:
    continuous_dataframe = dataframe[continuous_fields]

    # calculate z-scores:
    z_scores = (continuous_dataframe - continuous_dataframe.groupby(by = parameters['stock_field']).transform('mean')) / continuous_dataframe.groupby(by = parameters['stock_field']).transform('std')

    # drop the null ticker (not needed post groupby): 
    z_scores.drop(columns = [ parameters['stock_field'], parameters['calculation_field'] ], inplace = True)

    # rename the fields to indicate standardization:
    z_scores.columns = z_scores.columns + '_std'

    # drop original continuous fields # TODO: coming back after calculation checks:
    if parameters['drop_original_fields'] == True:
        continuous_fields.remove(parameters['stock_field'])
        dataframe.drop(columns = continuous_fields, inplace = True)

    # append the fields back into the core dataframe:
    z_scores = pd.concat([dataframe, z_scores], axis = 1)

    # remove the standardized target field:
    z_scores.drop(columns = z_scores.columns[z_scores.columns.str.contains('target')][1], inplace = True)

    # remove unnecessary items:
    del continuous_fields, continuous_dataframe

    return z_scores




In [227]:
df = catalog.load('combined_modeling_input')

[01/04/23 08:53:54] INFO     Loading data from 'combined_modeling_input' (CSVDataSet)...        ]8;id=913467;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=55869;file:///opt/anaconda3/envs/stock-classification/lib/python3.7/site-packages/kedro/io/data_catalog.py#344\344]8;;\

In [228]:
parameters = {'continuous_feature_cutoff' : 0.60,
              'stock_field' : 'ticker',
              'calculation_field' : 'close',
              'drop_original_fields' : True}

In [ ]:
##################################### - Function development HERE

In [3]:
def create_training_test_splits(dataframe: pd.DataFrame, parameters: dict) -> pd.DataFrame:

    '''Function that splits out training and test sets for machine learning
    Args:
        dataframe: pandas dataframe containing only the target field and the features to be used by the classifier
        parameters:
            test_ratio: proportion of samples in the dataframe to be used as a test set once the models are tuned and evaluated

    '''

In [212]:
##################################### - Testing functions HERE

In [229]:
# test run code chunk:

df = clean_stock_data(dataframe= df)

# create the standardized features:

df = standardize_continuous_features(dataframe = df, parameters = parameters)

In [231]:
# drop the standardized target field:
#df.columns[df.columns.str.contains('target')][0]

# separated out the target field:
df.columns[df.columns.str.contains('target')][0]

'target_20_days_ahead_ind'

In [232]:
df.columns

Index(['date', 'ticker', 'above_7_close_sma_ind', 'above_14_close_sma_ind',
       'above_21_close_sma_ind', 'cum_days_above_above_7_close_sma_ind',
       'cum_days_above_above_14_close_sma_ind',
       'cum_days_above_above_21_close_sma_ind', 'bol_range_pct',
       'target_20_days_ahead_ind', '14_close_sma_std',
       '14_close_sma_pct_diff_std', '14_close_std_std', '21_close_sma_std',
       '21_close_sma_pct_diff_std', '21_close_std_std', '7_close_sma_std',
       '7_close_sma_pct_diff_std', '7_close_std_std',
       'bol_pct_from_bottom_std', 'bol_pct_from_top_std', 'bol_range_std',
       'lower_bollinger_band_std', 'upper_bollinger_band_std'],
      dtype='object')